In [129]:
import numpy as np
import pandas as pd
import math
from typing import Tuple

pd.set_option('display.precision', 12)  # Increase decimal precision
pd.set_option('display.width', 300)     # Wider display
pd.set_option('display.max_columns', None)  # Show all column

In [130]:
# ------------------------------------------------------------
# Helper function 1: w_function
# Produce coefficient list of w_{n+1}(x) = Π (x - x_i)
# ------------------------------------------------------------
def w_function(x_points):
    coeff = np.array([1.0])  # start with polynomial 1
    for xi in x_points:
        coeff = np.convolve(coeff, np.array([1.0, -xi]))  # multiply by (x - xi)
    return coeff  # highest to lowest degree

In [131]:
# ------------------------------------------------------------
# Helper function 2: c_function
# Produce coefficient list of C_i(x) = w_{n+1}(x) / (x - x_i)
# ------------------------------------------------------------
def c_function(w_coeffs, xi):
    n = len(w_coeffs)
    c_coeffs = np.zeros(n - 1)
    remainder = 0.0
    # Synthetic division by (x - xi)
    for j in range(n - 1):
        if j == 0:
            c_coeffs[j] = w_coeffs[j]
        else:
            c_coeffs[j] = w_coeffs[j] + xi * c_coeffs[j - 1]
    remainder = w_coeffs[-1] + xi * c_coeffs[-1]
    # We expect remainder ≈ 0 if everything is correct
    return c_coeffs


# Lagrange Interpolation

## Algorithm

- Algorithm: Lagrange Interpolation

**Input:** $(x_0, y_0), (x_1, y_1), \dots, (x_n, y_n)$  with $x_i$ increasing and distinct.

**Output:** Coefficients of polynomial $L(x)$ passing through all given points.

- Steps

1. Compute the base polynomial $w_{n+1}(x) = \prod_{i=0}^{n} (x - x_i)$  
   → obtain coefficient list of $w_{n+1}(x)$.

2. For each $i = 0, 1, \dots, n$:
   - Compute $D_i = \dfrac{y_i}{\prod_{k=0, k \ne i}^{n} (x_i - x_k)}$
   - Compute $C_i(x) = \dfrac{w_{n+1}(x)}{(x - x_i)}$
   - Form $L_i(x) = D_i \cdot C_i(x)$

3. Sum all sub-polynomials:  
   $L(x) = \sum_{i=0}^{n} L_i(x)$

4. Return the coefficient list of $L(x)$ from lowest to highest degree.

In [132]:

# ------------------------------------------------------------
# Main function: lagrange_interpolation
# ------------------------------------------------------------
def lagrange_interpolation(points):
    x_points = np.array([p[0] for p in points], dtype=float)
    y_points = np.array([p[1] for p in points], dtype=float)
    n = len(points) - 1

    # Step 1: Compute w(x)
    w_coeffs = w_function(x_points)

    w_coeffs_df = pd.DataFrame({'Degree': list(range(len(w_coeffs)-1, -1, -1)), 'Coeff': w_coeffs})

    # Step 2: Compute each L_i(x)
    all_Li = []
    table_data = []

    for i in range(n + 1):
        xi, yi = x_points[i], y_points[i]
        
        # D_i calculation
        denom = np.prod([xi - xk for j, xk in enumerate(x_points) if j != i])
        D_i = yi / denom
        
        # C_i(x)
        C_i = c_function(w_coeffs, xi)
        
        # L_i(x) = D_i * C_i(x)
        L_i = D_i * C_i
        all_Li.append(L_i)
        
        # Store intermediate results
        table_data.append({
            'i': i,
            'x_i': xi,
            'y_i': yi,
            'denom': denom,
            'D_i': D_i,
            'C_i(x) coeffs': C_i.tolist(),
            'L_i(x) coeffs': L_i.tolist()
        })

    sub_lagrange_df = pd.DataFrame(table_data)

    # Step 3: Sum all L_i(x)
    max_len = max(len(Li) for Li in all_Li)
    L_coeffs = np.zeros(max_len)
    for Li in all_Li:
        L_coeffs[-len(Li):] += Li  # align degrees

    coeff_table = pd.DataFrame({
        'Degree': list(range(len(L_coeffs))),
        'Coeff': L_coeffs,
    })

    return w_coeffs_df, sub_lagrange_df, coeff_table

## Result

In [133]:
# ------------------------------------------------------------
# Example run
# ------------------------------------------------------------
#points = [(1, 17), (2, 17.5), (3, 76), (4, 210.5), (7, 1970)]
points = [(1.2, 0.892), (1.5, 1.179), (1.7, 1.358), (1.8, 1.445), (2.1, 1.688), (2.3, 1.839)]
w_coeffs_df, sub_lagrange_df, coeff_table = lagrange_interpolation(points)

In [140]:
w_coeffs_df.style.hide(axis="index")

Degree,Coeff
6,1.000000
5,-10.600000
4,46.420000
3,-107.472000
2,138.698100
1,-94.569660
0,26.603640


In [138]:
sub_lagrange_df.style.hide(axis="index")

i,x_i,y_i,denom,D_i,C_i(x) coeffs,L_i(x) coeffs
0,1.200000,0.892000,-0.089100,-10.011223,"[1.0, -9.400000000000002, 35.14, -65.304, 60.33330000000001, -22.16969999999999]","[-10.011223344556676, 94.10549943883278, -351.7943883277216, 653.7729292929292, -604.0101414141413, 221.94581818181805]"
1,1.500000,1.179000,0.008640,136.458333,"[1.0, -9.100000000000001, 32.769999999999996, -58.317000000000014, 51.222599999999986, -17.735760000000028]","[136.45833333333334, -1241.7708333333337, 4471.739583333333, -7957.840625000003, 6989.750624999999, -2420.1922500000037]"
2,1.700000,1.358000,-0.002400,-565.833333,"[1.0, -8.900000000000002, 31.29, -54.27900000000001, 46.4238, -15.649200000000008]","[-565.8333333333329, 5035.916666666664, -17704.924999999985, 30712.867499999982, -26268.13349999998, 8854.838999999998]"
3,1.800000,1.445000,0.002700,535.185185,"[1.0, -8.8, 30.58, -52.42800000000001, 44.32769999999999, -14.779800000000009]","[535.1851851851849, -4709.629629629628, 16365.962962962953, -28058.68888888888, 23723.528333333317, -7909.93]"
4,2.100000,1.688000,-0.012960,-130.246914,"[1.0, -8.500000000000002, 28.569999999999997, -47.475000000000016, 39.00059999999998, -12.668400000000048]","[-130.246913580247, 1107.0987654320998, -3721.1543209876563, 6183.472222222228, -5079.707777777778, 1650.0200000000073]"
5,2.300000,1.839000,0.052800,34.829545,"[1.0, -8.3, 27.330000000000002, -44.61300000000001, 36.0882, -11.5668]","[34.82954545454554, -289.085227272728, 951.8914772727296, -1553.8505113636404, 1256.9356022727304, -402.86638636363733]"


In [139]:
coeff_table.style.hide(axis="index")

Degree,Coeff
0,0.381594
1,-3.364759
2,11.720314
3,-20.267374
4,18.363141
5,-6.183818
